# Geny Experimental Notebook
---

## Evaluated tools

| Tool | Version |
|------|---------|
| Geny | [Latest](https://github.com/0xTCG/geny) |
| T1K  | [v1.0.5](https://github.com/mourisl/T1K) (c02398b17) |
| PING | Please see blow |


## Simulations

### Generation
Paired-end simulations were generated using script:
[analysis.py](analysis.py)
### Run

Geny: python [geny.py](../geny.py) file.fa -c 20

T1K: 

```bash
cat file.fa | paste - - - - | \
    tee >(cut -f 1-2 | sed 's/^>/@/' | awk '{print $1; print $3; print "+"; print "HHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHH"}' > file.fq1) | \
    cut -f3-4 |        sed 's/^>/@/' | awk '{print $1; print $3; print "+"; print "HHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHH"}' > file.fq2
time ./run-t1k -1 file.fq1 file.fq2 --od file -t 8 --preset kir-wgs -f kiridx/kiridx_dna_seq.fa
```

## HPRC Samples
### HPRC data acquisition
We evaluated the following 40 HPRC samples:
| HG00438 | HG01175 | HG02148 | HG00621 | HG01243 |
|---------|---------|---------|---------|---------|
| HG02257 | HG00673 | HG01258 | HG02572 | HG00733 |
| HG01358 | HG02622 | HG00735 | HG01361 | HG02630 |
| HG00741 | HG01891 | HG02717 | HG01071 | HG01928 |
| HG02723 | HG01106 | HG01952 | HG02818 | HG01109 |
| HG01978 | HG02886 | HG02055 | HG03098 | HG03579 |
| HG02080 | HG03453 | NA18906 | HG02145 | HG03486 |
| NA19240 | HG03492 | HG03540 | NA20129 | HG03516 |

The HPRC CRAM / CRAI files are acquired from the following links:
- [HPRC](https://s3-us-west-2.amazonaws.com/human-pangenomics/index.html?prefix=working/HPRC/)
- [HPRC_PLUS](https://s3-us-west-2.amazonaws.com/human-pangenomics/index.html?prefix=working/HPRC_PLUS/)


### Data preparation for T1K
1. Download and unzip Gencode reference:
   ```bash
   wget https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_46/gencode.v46.chr_patch_hapl_scaff.annotation.gtf.gz
   gunzip gencode.v46.chr_patch_hapl_scaff.annotation.gtf.gz
2. Process the Gencode file:
   ```bash
   cat gencode.v46.chr_patch_hapl_scaff.annotation.gtf | \
   grep -v '^KV' | \
   sed 's/GL949747.2/chr19_GL949747v2_alt/' | \
   sed 's/KI270890.1/chr19_KI270890v1_alt/' | \
   sed 's/KI270921.1/chr19_KI270921v1_alt/' | \
   sed 's/KI270930.1/chr19_KI270930v1_alt/' > \
   gencode.v46.chr_patch_hapl_scaff.annotation_.gtf
3. Build KIR index using IPD-KIR 2.12:
   ```bash
   perl t1k-build.pl -o kiridx -d kiridx/kir.dat \
   -g gencode.v46.chr_patch_hapl_scaff.annotation_.gtf \
   --partial-intron-noseq

### Benchmark each tool

To benchmark Geny and T1K on hprc data, we used the following command:
##### Geny:

parallel --bar -j2 'python [geny.py](../geny.py) {} > results_validation/{/.}.log 2>&1' ::: data/hprc/*.cram

##### T1K: 
```bash
parallel --bar -j2 './run-t1k -b {} -c kiridx/kiridx_dna_coord.fa --od results --abnormalUnmapFlag -t 8 --preset kir-wgs -f kiridx/kiridx_dna_seq.fa > results/{/.}.log 2>&1' ::: data/hprc/*.bam
```

### Running HPRC samples on the latest version of PING
#### PING repository: https://github.com/wesleymarin/PING branch: `wgs-snakemake`

#### Inputs
1. Download 40 `HPRC` + `HPRC_PLUS` CRAM files
2. Download the published BED file for extraction of KIR reads. Published along with `doi:10.1111/tan.14949` (Table S3)

#### Extract relevant reads from the CRAM files using the following bash script. Input: CRAM. Output: bgzipped fastqs

```bash
#!/bin/bash
cram=$1

fname=${cram##*/}
fname=${fname%%.*}
out_dir=fastqs/hprc
echo $fname
fastq1="$out_dir"/"$fname"_1.fq
fastq2="$out_dir"/"$fname"_2.fq

bed=kir-PING-hg38.bed
REF=GRCh38_full_analysis_set_plus_decoy_hla.fa

samtools view -hu -L $bed -T $REF $cram | samtools collate -@ $SLURM_CPUS_PER_TASK -Ou - | samtools fastq -@ $SLURM_CPUS_PER_TASK -1 ${fastq1} -2 ${fastq2} -0 /dev/null -s /dev/null -n -

# --- bgzip --- #
echo "bgzip -@ $SLURM_CPUS_PER_TASK ${fastq1}"
bgzip -@ $SLURM_CPUS_PER_TASK ${fastq1}
echo "bgzip -@ $SLURM_CPUS_PER_TASK ${fastq2}"
bgzip -@ $SLURM_CPUS_PER_TASK ${fastq2}

```
### Download PING --- Follow instructions from https://github.com/wesleymarin/PING/tree/wgs_snakemake
Git clone the repo:
```bash
git clone https://github.com/Hollenbach-lab/PING.git
cd ./PING
```

### Running PING
#### Install R dependencies (`4.3` version of R)
``` bash
# R dependency install command
install.packages(c("data.table","plotly","stringr","pryr","gtools","R.utils","Parallel"),dependencies = T)
```

### Run PING from within the cloned project
(PING produces an `extractedFastq` directory for all samples in the input directory (input dir itself contains reads extracted from the KIR region). We use one such directory from an initial run which failed on a single sample.)

``` bash
# As mentioned within the README, we change the following lines in PING_run.R and run the entire script in rstudio top to bottom
rawFastqDirectory <- <fastq dir> # can be set to raw sequence or extractedFastq directory
fastqPattern <- '_KIR_' # use '_KIR_' to find already extracted files, otherwise use 'fastq' or whatever fits your data
threads <- 10
resultsDirectory <- <output dir> # Set the master results directory (all pipeline output will be recorded here)
shortNameDelim <- '_' 
```
This script pauses when it generates plots for thresholding copy numbers for all KIR genes. Set the thresholds and resume running.


Alternatively, one could enter the threshold values into `<output dir>/manualCopyThresholds.csv` and rerun the entire script with after setting the last flag of line 94 of `PING_run.R` to `True`, which is 

``` bash
sampleList <- ping_copy.manual_threshold(sampleList=sampleList,resultsDirectory=outDir$path,use.threshFile = Tb) # this function sets copy thresholds
```

The manualCopyThresholds.csv file for all 40 smaples is given below

<!-- "","0-1","1-2","2-3","3-4","4-5","5-6"
"KIR3DP1",0.15,0.35,0.6,0.8,NA,NA
"KIR2DS5",0.2,0.6,1.1,NA,NA,NA
"KIR2DL3",0.2,0.6,1,NA,NA,NA
"KIR2DP1",0.3,0.6,1.2,NA,NA,NA
"KIR2DS3",0.2,0.7,1.2,NA,NA,NA
"KIR2DS2",0.18,0.5,0.8,NA,NA,NA
"KIR2DL4",0.2,0.7,1.2,1.7,NA,NA
"KIR3DL3",0,0,NA,NA,NA,NA
"KIR3DL1",0.2,0.6,0.7,1,NA,NA
"KIR3DS1",0.2,0.6,0.9,1.1,NA,NA
"KIR2DL2",0.15,0.28,0.45,1.1,NA,NA
"KIR3DL2",0.2,1.1,1.6,NA,NA,NA
"KIR2DS4",0.2,0.8,1.2,NA,NA,NA
"KIR2DL1",0.2,0.47,0.59,0.8,1.2,NA
"KIR2DS1",0.15,0.4,NA,NA,NA,NA
"KIR2DL5",0.15,0.5,0.8,1.1,NA,NA -->
| Gene     | 0-1   | 1-2   | 2-3   | 3-4   | 4-5   | 5-6 |
|----------|-------|-------|-------|-------|-------|-----|
| KIR3DP1  | 0.15  | 0.35  | 0.6   | 0.8   | NA    | NA  |
| KIR2DS5  | 0.2   | 0.6   | 1.1   | NA    | NA    | NA  |
| KIR2DL3  | 0.2   | 0.6   | 1     | NA    | NA    | NA  |
| KIR2DP1  | 0.3   | 0.6   | 1.2   | NA    | NA    | NA  |
| KIR2DS3  | 0.2   | 0.7   | 1.2   | NA    | NA    | NA  |
| KIR2DS2  | 0.18  | 0.5   | 0.8   | NA    | NA    | NA  |
| KIR2DL4  | 0.2   | 0.7   | 1.2   | 1.7   | NA    | NA  |
| KIR3DL3  | 0     | 0     | NA    | NA    | NA    | NA  |
| KIR3DL1  | 0.2   | 0.6   | 0.7   | 1     | NA    | NA  |
| KIR3DS1  | 0.2   | 0.6   | 0.9   | 1.1   | NA    | NA  |
| KIR2DL2  | 0.15  | 0.28  | 0.45  | 1.1   | NA    | NA  |
| KIR3DL2  | 0.2   | 1.1   | 1.6   | NA    | NA    | NA  |
| KIR2DS4  | 0.2   | 0.8   | 1.2   | NA    | NA    | NA  |
| KIR2DL1  | 0.2   | 0.47  | 0.59  | 0.8   | 1.2   | NA  |
| KIR2DS1  | 0.15  | 0.4   | NA    | NA    | NA    | NA  |
| KIR2DL5  | 0.15  | 0.5   | 0.8   | 1.1   | NA    | NA  |


### Evaluation
We evaluated the results against the BAKIR annotations located in [bakir-annotations](ground/bakir-annotations), which is summarized with:[get_hprc_ground.py](ground/get_hprc_ground.py) The ground truth csv file was  [hprc_gt_0801.csv](ground/hprc_gt_0801.csv), and we used [summarize_hprc.py](summarize_hprc.py) for HPRC result summarization.
